In [1]:
import cv2
import torch
from yoloDetect import *

YOLOv5 🚀 2023-3-7 Python-3.9.7 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24256MiB)



#### Select a video for IoM visualization

In [5]:
# Raw video path, on which you would like to visualize IoM 

video_path = ''# Put the path here
capture = cv2.VideoCapture(video_path)

#### Initialize Yolo and create a holder for door bboxes

In [3]:
device = '1'
yolo = yoloDetect(device=select_device(f"cuda:{device}"))
door_weights = 'runs/train/door_model/weights/best.pt'
person_weights = 'yolov5s.pt'

# Initializing door/person Yolo
door_yolo = DetectMultiBackend(
    weights=door_weights, device=select_device(f"cuda:{device}"))
person_yolo = DetectMultiBackend( weights=person_weights, device=select_device(f"cuda:{device}"))

door_bbox = np.zeros(0)

YOLOv5 🚀 2023-3-7 Python-3.9.7 torch-1.13.1 CUDA:1 (NVIDIA GeForce RTX 3090 Ti, 24256MiB)

YOLOv5 🚀 2023-3-7 Python-3.9.7 torch-1.13.1 CUDA:1 (NVIDIA GeForce RTX 3090 Ti, 24256MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
YOLOv5 🚀 2023-3-7 Python-3.9.7 torch-1.13.1 CUDA:1 (NVIDIA GeForce RTX 3090 Ti, 24256MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


#### Run Yolos for door and person detection, compute IoM

In [6]:
%matplotlib notebook

door_bbox = None
conf_thres = 0.4
door_conf_thres = 0.6
person_conf_thres = 0.5
door_detect = False
zone_thresh = 20
line_thickness = 5
person_bbox = np.zeros((0, 4))
fig, ax = plt.subplots()
ret, frame = capture.read()
plt.ion() 

while ret:
    img = frame.copy()
    img = img[..., ::-1]
    if not door_detect:
        door_bbox = yolo(
                    door_yolo,
                    img,
                    img_size=(416, 416),
                    scale_with=(1080, 1920),
                    conf_thres=door_conf_thres
        )[:, 1:]
        
        if not len(door_bbox):
            print("No detections in frame")
            ret, frame = capture.read()
            continue
        else:
            door_detect=True
            door_bbox = door_bbox[0]

            x, y, x_w, y_h = (
                door_bbox[0] - zone_thresh,
                door_bbox[1],
                door_bbox[2] + zone_thresh,
                door_bbox[3] + zone_thresh,
            )
            x, x_w, y, y_h = round_box(x, x_w, y, y_h, imgsz=img.shape[:2])
            crop_size = (y, y_h, x, x_w)
            imgsz = (y_h - y, x_w - x)
            p1_door = (int(door_bbox[0]), int(door_bbox[1]))
            p2_door = (int(door_bbox[2]), int(door_bbox[3]))

    if len(door_bbox):
        # Engage person Yolo
        bboxes = yolo(
                    person_yolo,
                    img,
                    img_size=imgsz,
                    crop_size=crop_size,
                    scale_with=imgsz,
                    conf_thres=person_conf_thres
                ).astype(int)
                
        # Put door rectangles on frame 
        cv2.rectangle(
                    img[..., ::-1],
                    p1_door,
                    p2_door,
                    (0, 0, 255),
                    thickness=line_thickness,
                    lineType=cv2.LINE_AA,
                )
        person_bbox = bboxes[np.where(bboxes[:, 0] == 0)[0], 1:]
        
        if len(bboxes):
            for bbox_person in person_bbox[:1]:
        
                bbox_person[1] += crop_size[0]
                bbox_person[3] += crop_size[0]
                bbox_person[0] += crop_size[2]
                bbox_person[2] += crop_size[2]
                
                p1 = (int(bbox_person[0]), int(bbox_person[1]))
                p2 = (int(bbox_person[2]), int(bbox_person[3]))
                cv2.rectangle(img[..., ::-1], p1, p2, (255, 0, 0), thickness=line_thickness, lineType=cv2.LINE_AA)
                iom = box_iom(torch.tensor([bbox_person]), torch.tensor([door_bbox]))
                cv2.putText(
                    img[..., ::-1],
                    f" IOM {round(iom.item(), 3)}",
                    (10, 200),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    2,
                    (255,255,255),
                    thickness=5,
                    lineType=cv2.LINE_AA)
    
    # plotting the video for IoM visualization
    ax.imshow(img)
    plt.show()
    fig.canvas.draw() 
    ax.cla()
    
    ret, frame = capture.read()